In [1]:
import os
import pathlib

import numpy as np
import pandas as pd

In [2]:
lfc = pd.read_csv(
    '../data/standardized_data/result_logfc_matrix_2020_08_27.csv')
lfc.head()

,Rv_ID,PE35_KO_vs_mbio_H37Rv,PPE68_KO_vs_mbio_H37Rv,Rv0950c_KO_vs_CB_WT,Rv0954_KO_vs_RJ_WT,Rv1096_KO_vs_CB_WT,Rv3005c_KO_day32_vs_dejesus_H37Rv_day32,Rv3594_KO_vs_Rubin_FLUTE_WT,Rv3684_KO_vs_CB_WT,Rv3717_KO_vs_Rubin_FLUTE_WT,...,zhang_AA_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_DETA-NO_pH_7.0_vs_zhang_pH_7.0_no_NO_control,zhang_Fe_1.5mM_vs_zhang_Fe_450uM,zhang_Trp_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_Tyloxapol_pH_6.5_vs_zhang_Tyloxapol_pH_4.5,zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45,zhang_wt_mouse_d10_vs_zhang_input_library,zhang_wt_mouse_d45_vs_zhang_input_library
0,Rv0001,-0.41,0.01,0.00,0.00,0.00,3.12,-0.06,0.00,-0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Rv0002,3.28,2.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,Rv0003,-0.03,-0.08,0.03,-1.20,0.39,3.83,0.31,0.17,-0.62,...,-2.93,-0.31,1.09,-2.13,0.51,0.41,1.30,-0.49,-1.26,-2.25
3,Rv0004,-3.81,-3.39,-2.60,-6.83,1.54,0.00,1.79,1.08,4.91,...,-3.56,0.00,1.96,-3.56,0.00,0.00,0.00,0.70,-2.62,-2.62
4,Rv0005,-0.22,0.14,0.13,-0.08,0.27,-2.38,0.70,-0.71,-0.37,...,-1.47,-0.89,-2.36,-1.89,2.90,2.20,-4.05,0.30,0.69,-3.55


In [3]:
qval = pd.read_csv(
    '../data/standardized_data/result_qval_matrix_2020_08_27.csv')
qval.head()

,Rv_ID,PE35_KO_vs_mbio_H37Rv,PPE68_KO_vs_mbio_H37Rv,Rv0950c_KO_vs_CB_WT,Rv0954_KO_vs_RJ_WT,Rv1096_KO_vs_CB_WT,Rv3005c_KO_day32_vs_dejesus_H37Rv_day32,Rv3594_KO_vs_Rubin_FLUTE_WT,Rv3684_KO_vs_CB_WT,Rv3717_KO_vs_Rubin_FLUTE_WT,...,zhang_AA_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_DETA-NO_pH_7.0_vs_zhang_pH_7.0_no_NO_control,zhang_Fe_1.5mM_vs_zhang_Fe_450uM,zhang_Trp_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_Tyloxapol_pH_6.5_vs_zhang_Tyloxapol_pH_4.5,zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45,zhang_wt_mouse_d10_vs_zhang_input_library,zhang_wt_mouse_d45_vs_zhang_input_library
0,Rv0001,1.0,1.0,1.0,1.00000,1.0,0.73174,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.0,1.00000,1.0,1.0,1.0,1.00000
1,Rv0002,1.0,1.0,1.0,1.00000,1.0,1.00000,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.0,1.00000,1.0,1.0,1.0,1.00000
2,Rv0003,1.0,1.0,1.0,0.43844,1.0,0.47324,1.0,1.0,1.0,...,1.0,1.0,0.68823,1.0,1.0,0.99313,1.0,1.0,1.0,0.06052
3,Rv0004,1.0,1.0,1.0,1.00000,1.0,1.00000,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.0,1.00000,1.0,1.0,1.0,0.81749
4,Rv0005,1.0,1.0,1.0,1.00000,1.0,0.62111,1.0,1.0,1.0,...,1.0,1.0,1.00000,1.0,1.0,0.70716,1.0,1.0,1.0,0.37106


In [4]:
assert (lfc.columns == qval.columns).all()

In [5]:
lfc.isna().sum()

Rv_ID                                           0
PE35_KO_vs_mbio_H37Rv                          65
PPE68_KO_vs_mbio_H37Rv                         65
Rv0950c_KO_vs_CB_WT                            65
Rv0954_KO_vs_RJ_WT                             65
                                               ..
zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5    65
zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10    65
zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45    65
zhang_wt_mouse_d10_vs_zhang_input_library      65
zhang_wt_mouse_d45_vs_zhang_input_library      65
Length: 65, dtype: int64

In [6]:
qval.isna().sum()

Rv_ID                                           0
PE35_KO_vs_mbio_H37Rv                          65
PPE68_KO_vs_mbio_H37Rv                         65
Rv0950c_KO_vs_CB_WT                            65
Rv0954_KO_vs_RJ_WT                             65
                                               ..
zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5    65
zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10    65
zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45    65
zhang_wt_mouse_d10_vs_zhang_input_library      65
zhang_wt_mouse_d45_vs_zhang_input_library      65
Length: 65, dtype: int64

In [7]:
lfc[lfc['PE35_KO_vs_mbio_H37Rv'].isna() == True]['Rv_ID'].unique()

array(['Rv0061c', 'Rv0064A', 'Rv0078B', 'Rv0157A', 'Rv0186Ac', 'Rv0229Ac',
       'Rv0277Ac', 'Rv0397A', 'Rv0456B', 'Rv0530Ac', 'Rv0616A', 'Rv0691A',
       'Rv0744Ac', 'Rv0959A', 'Rv1144Ac', 'Rv1366A', 'Rv1735Ac', 'Rv1792',
       'Rv1792A', 'Rv1954A', 'Rv1962A', 'Rv1982A', 'Rv1984A', 'Rv1991A',
       'Rv2023A', 'Rv2063A', 'Rv2077Bc', 'Rv2098c', 'Rv2099c', 'Rv2142A',
       'Rv2190Ac', 'Rv2231Ac', 'Rv2231B', 'Rv2237A', 'Rv2255A', 'Rv2274A',
       'Rv2309Xc', 'Rv2386A', 'Rv2395A', 'Rv2395B', 'Rv2427Ac', 'Rv2468A',
       'Rv2512A', 'Rv2548A', 'Rv2643A', 'Rv2801A', 'Rv2828A', 'Rv2856A',
       'Rv2856B', 'Rv2862A', 'Rv2873A', 'Rv2964B', 'Rv2975A', 'Rv3018Bc',
       'Rv3032A', 'Rv3098A', 'Rv3098B', 'Rv3136A', 'Rv3190A', 'Rv3202A',
       'Rv3372A', 'Rv3666B', 'Rv3697A', 'Rv3749Ac', 'Rv3857Ac'],
      dtype=object)

In [8]:
qval[qval['PE35_KO_vs_mbio_H37Rv'].isna() == True]['Rv_ID'].unique()

array(['Rv0061c', 'Rv0064A', 'Rv0078B', 'Rv0157A', 'Rv0186Ac', 'Rv0229Ac',
       'Rv0277Ac', 'Rv0397A', 'Rv0456B', 'Rv0530Ac', 'Rv0616A', 'Rv0691A',
       'Rv0744Ac', 'Rv0959A', 'Rv1144Ac', 'Rv1366A', 'Rv1735Ac', 'Rv1792',
       'Rv1792A', 'Rv1954A', 'Rv1962A', 'Rv1982A', 'Rv1984A', 'Rv1991A',
       'Rv2023A', 'Rv2063A', 'Rv2077Bc', 'Rv2098c', 'Rv2099c', 'Rv2142A',
       'Rv2190Ac', 'Rv2231Ac', 'Rv2231B', 'Rv2237A', 'Rv2255A', 'Rv2274A',
       'Rv2309Xc', 'Rv2386A', 'Rv2395A', 'Rv2395B', 'Rv2427Ac', 'Rv2468A',
       'Rv2512A', 'Rv2548A', 'Rv2643A', 'Rv2801A', 'Rv2828A', 'Rv2856A',
       'Rv2856B', 'Rv2862A', 'Rv2873A', 'Rv2964B', 'Rv2975A', 'Rv3018Bc',
       'Rv3032A', 'Rv3098A', 'Rv3098B', 'Rv3136A', 'Rv3190A', 'Rv3202A',
       'Rv3372A', 'Rv3666B', 'Rv3697A', 'Rv3749Ac', 'Rv3857Ac'],
      dtype=object)

In [9]:
print(lfc.shape, qval.shape)
lfc = lfc.dropna(axis=0)
qval = qval.dropna(axis=0)
print(lfc.shape, qval.shape)

(4055, 65) (4055, 65)
(3971, 65) (3971, 65)


In [10]:
assert (lfc.columns == qval.columns).all()
assert (lfc.Rv_ID == qval.Rv_ID).all()

In [11]:
mcbwser = pd.read_excel(pathlib.Path.cwd().parents[0].joinpath(
    'data', 'annotations', 'Mycobacterium_tuberculosis_H37Rv_txt_v3.xlsx'))
mcbwser.head()

,Refseq_ID,Feature,Rv_ID,Name,Function,Product,Comments,Functional_Category
0,NC_000962.3,CDS,Rv3728,Rv3728,"Unknown, but seems involved in efflux system (...",Probable conserved two-domain membrane protein,"Rv3728, (MTV025.076), len: 1065 aa. Probable c...",cell wall and cell processes
1,NC_000962.3,CDS,Rv3729,Rv3729,Function unknown; probably involved in cellula...,Possible transferase,"Rv3729, (MTV025.077), len: 776 aa. Conserved h...",intermediary metabolism and respiration
2,NC_000962.3,CDS,Rv3730c,Rv3730c,Function unknown,Conserved hypothetical protein,"Rv3730c, (MTV025.078c), len: 346 aa. Conserved...",conserved hypotheticals
3,NC_000962.3,CDS,Rv3760,Rv3760,Unknown,Possible conserved membrane protein,"Rv3760, (MTV025.108), len: 100 aa. Possible co...",cell wall and cell processes
4,NC_000962.3,CDS,Rv3732,Rv3732,Function unknown,Conserved protein,"Rv3732, (MTV025.080), len: 352 aa. Conserved p...",conserved hypotheticals


In [12]:
mcbwser[mcbwser['Rv_ID'].duplicated()]

,Refseq_ID,Feature,Rv_ID,Name,Function,Product,Comments,Functional_Category
1339,NC_000962.3,promoter,Rv3571,kshB,Predicted to be involved in lipid catabolism,Reductase component of 3-ketosteroid-9-alpha-h...,"Rv3571, (MTCY06G11.18), len: 358 aa. kshB, red...",intermediary metabolism and respiration
1340,NC_000962.3,CDS,Rv3571,kshB,Predicted to be involved in lipid catabolism,Reductase component of 3-ketosteroid-9-alpha-h...,"Rv3571, (MTCY06G11.18), len: 358 aa. kshB, red...",intermediary metabolism and respiration
1572,NC_000962.3,CDS,Rv0148,Rv0148,Function unknown; possibly involved in cellula...,Probable short-chain type dehydrogenase/reductase,"Rv0148, (MTCI5.22), len: 286 aa. Probable shor...",intermediary metabolism and respiration
2410,NC_000962.3,-10_signal,MTB000063,mcr11,Unknown,Putative small regulatory RNA,"mcr11, putative small regulatory RNA (See DiCh...",stable RNAs
2411,NC_000962.3,-35_signal,MTB000063,mcr11,Unknown,Putative small regulatory RNA,"mcr11, putative small regulatory RNA (See DiCh...",stable RNAs
2414,NC_000962.3,-10_signal,Rv1219c,Rv1219c,Involved in transcriptional mechanism.,Probable transcriptional regulatory protein,"Rv1219c, (MTCI61.02c), len: 212 aa. Probable t...",regulatory proteins
2415,NC_000962.3,-35_signal,Rv1219c,Rv1219c,Involved in transcriptional mechanism.,Probable transcriptional regulatory protein,"Rv1219c, (MTCI61.02c), len: 212 aa. Probable t...",regulatory proteins
2962,NC_000962.3,-10_signal,Rv1964,yrbE3A,Unknown,Conserved hypothetical integral membrane prote...,"Rv1964, (MTV051.02), len: 265 aa. YrbE3A, hypo...","virulence, detoxification, adaptation"
2963,NC_000962.3,CDS,Rv1964,yrbE3A,Unknown,Conserved hypothetical integral membrane prote...,"Rv1964, (MTV051.02), len: 265 aa. YrbE3A, hypo...","virulence, detoxification, adaptation"
2991,NC_000962.3,promoter,Rv1994c,cmtR,Represses transcription from the CMT operator-...,Metal sensor transcriptional regulator CmtR (A...,"Rv1994c, (MTCY39.25), len: 118 aa. CmtR, trans...",regulatory proteins


In [13]:
mcbwser = mcbwser.drop_duplicates(subset=['Rv_ID'])

Creating a new data frame by merging using the RvID

In [14]:
lfc_mb = pd.merge(lfc, mcbwser[['Rv_ID', 'Functional_Category']],
                  how='left', on='Rv_ID')
qval_mb = pd.merge(qval, mcbwser[['Rv_ID', 'Functional_Category']],
                   how='left', on='Rv_ID')

In [15]:
lfc_mb.head()

,Rv_ID,PE35_KO_vs_mbio_H37Rv,PPE68_KO_vs_mbio_H37Rv,Rv0950c_KO_vs_CB_WT,Rv0954_KO_vs_RJ_WT,Rv1096_KO_vs_CB_WT,Rv3005c_KO_day32_vs_dejesus_H37Rv_day32,Rv3594_KO_vs_Rubin_FLUTE_WT,Rv3684_KO_vs_CB_WT,Rv3717_KO_vs_Rubin_FLUTE_WT,...,zhang_DETA-NO_pH_7.0_vs_zhang_pH_7.0_no_NO_control,zhang_Fe_1.5mM_vs_zhang_Fe_450uM,zhang_Trp_Rescue_vs_zhang_in_vitro_control_Rescue,zhang_Tyloxapol_pH_6.5_vs_zhang_Tyloxapol_pH_4.5,zhang_Tyloxapol_pH_6.5_vs_zhang_pcit_pH_4.5,zhang_mhcii_mouse_d10_vs_zhang_wt_mouse_d10,zhang_mhcii_mouse_d45_vs_zhang_wt_mouse_d45,zhang_wt_mouse_d10_vs_zhang_input_library,zhang_wt_mouse_d45_vs_zhang_input_library,Functional_Category
0,Rv0001,-0.41,0.01,0.00,0.00,0.00,3.12,-0.06,0.00,-0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,information pathways
1,Rv0002,3.28,2.38,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,information pathways
2,Rv0003,-0.03,-0.08,0.03,-1.20,0.39,3.83,0.31,0.17,-0.62,...,-0.31,1.09,-2.13,0.51,0.41,1.30,-0.49,-1.26,-2.25,information pathways
3,Rv0004,-3.81,-3.39,-2.60,-6.83,1.54,0.00,1.79,1.08,4.91,...,0.00,1.96,-3.56,0.00,0.00,0.00,0.70,-2.62,-2.62,conserved hypotheticals
4,Rv0005,-0.22,0.14,0.13,-0.08,0.27,-2.38,0.70,-0.71,-0.37,...,-0.89,-2.36,-1.89,2.90,2.20,-4.05,0.30,0.69,-3.55,information pathways


In [16]:
lfc_mb['Functional_Category'].value_counts(dropna=False)

conserved hypotheticals                    1020
intermediary metabolism and respiration     933
cell wall and cell processes                771
lipid metabolism                            270
information pathways                        242
virulence, detoxification, adaptation       220
regulatory proteins                         197
PE/PPE                                      162
insertion seqs and phages                   142
unknown                                      14
Name: Functional_Category, dtype: int64

In [81]:
lfc_mb.shape

(3971, 66)

In [82]:
qval_mb['Functional_Category'].value_counts(dropna=False)

conserved hypotheticals                    1020
intermediary metabolism and respiration     933
cell wall and cell processes                771
lipid metabolism                            270
information pathways                        242
virulence, detoxification, adaptation       220
regulatory proteins                         197
PE/PPE                                      162
insertion seqs and phages                   142
unknown                                      14
Name: Functional_Category, dtype: int64

In [83]:
qval_mb.shape

(3971, 66)

In [84]:
lfc_mb.to_csv('../data/standardized_data/cleaned_ML/lfc_mb.csv', index=False)
qval_mb.to_csv('../data/standardized_data/cleaned_ML/qval_mb.csv', index=False)

In [85]:
lfc_mb_filt = lfc_mb[~lfc_mb['Functional_Category'].isin(
    ['conserved hypotheticals', 'unknown'])]
qval_mb_filt = qval_mb[~qval_mb['Functional_Category'].isin(
    ['conserved hypotheticals', 'unknown'])]

In [86]:
assert (lfc_mb_filt.Rv_ID == qval_mb_filt.Rv_ID).all()

In [87]:
lfc_mb_filt.to_csv(
    '../data/standardized_data/cleaned_ML/lfc_mb_filt.csv', index=False)
qval_mb_filt.to_csv(
    '../data/standardized_data/cleaned_ML/qval_mb_filt.csv',  index=False)

In [88]:
data_cols = [col for col in qval_mb.columns if col not in [
    'Rv_ID', 'Functional_Category']]

In [89]:
bin_matrix_lfc = lfc_mb[data_cols].applymap(lambda x: x >= 1 or x <= -1)
bin_matrix_qval = qval_mb[data_cols].applymap(lambda x: x <= 0.05)

In [90]:
bin_matrix = bin_matrix_lfc & bin_matrix_qval
bin_matrix = bin_matrix.astype(int)

In [91]:
bin_matrix['Rv_ID'] = qval_mb['Rv_ID']
bin_matrix['Functional_Category'] = qval_mb['Functional_Category']

In [92]:
bin_matrix = bin_matrix[['Rv_ID'] +
                        [col for col in bin_matrix.columns if col != 'Rv_ID']]

In [93]:
bin_matrix.to_csv('../data/standardized_data/cleaned_ML/bin_mb.csv',  index=False)

In [94]:
bin_matrix_filt = bin_matrix[~bin_matrix['Functional_Category'].isin(
    ['conserved hypotheticals', 'unknown'])]

In [95]:
bin_matrix_filt.to_csv(
    '../data/standardized_data/cleaned_ML/bin_mb_filt.csv', index=False)